# HW5 Technical: Blender Scripting

For this assignment, you'll be trying out the scripting options in blender, and creating a function that can predict the bunny and light's coordinate frames from an image:
* TODO1 [Blender]: use the Blender scripting tool to rotate the bunny and light and render an image.
* TODO2 [Blender]: use the Blender scripting tool to render a batch of images.
* TODO3 [Blender]: for each image in the batch, record the corresponding orientation (pose) of the light and bunny in a json file.
* TODO4 [notebook]: run the optimizer given, and generate a prediction.
* TODO5 [Blender]: take the prediction, and use the Blender script to generate a render corresponding to that prediction.


Before running this notebook , make sure you've run 

```conda install pytorch==1.10.1 torchvision==0.11.2 torchaudio==0.10.1 cpuonly -c pytorch```

in your anaconda environment. If you forgot to do this, close jupyter notebook and run the installation command first.

Also note again that you will not need to do any coding in the notebook for this assignment. Run the cells and verify that the output is correct, and respond to the short response questions.

In [ ]:
#DO NOT EDIT
import os, sys, glob, json
from typing import Any, Dict

import cv2
import numpy as np
import torch
import tqdm
from matplotlib import pyplot as plt

%load_ext autoreload
%autoreload 2

def show_image(filename:str, height:float=4, width:float=4,label:str=None):
    img=cv2.imread(filename)
    img=cv2.cvtColor(img, cv2.COLOR_BGR2RGB)        
    plt.gca().grid(None)
    plt.gca().axis("off")
    plt.imshow(img)
    if label is not None:
        plt.gca().set_title(label)
    plt.gcf().set_size_inches((width,height))
    plt.show()

In [ ]:
# Before running this cell, finish the TODO1 sections in the Blender file.
show_image("images/todo1.png")

# Short Response
* What are the rotation angles z_bunny and z_light for `todo1.png`? (Check the code in the Blender file that resulted in this output.)

```Your answer here```


In [ ]:
# Before running this cell, finish the TODO2 and TODO3 sections in the Blender file.
def show_images(parameter_dict:Dict[str,Any], scale:float=1):
    num_images=len(parameter_dict)
    num_columns=int(np.ceil(num_images**0.5))
    num_rows=(num_images+num_columns-1)//num_columns
    
    fig,axes=plt.subplots(num_rows,num_columns,gridspec_kw={'wspace':0, 'hspace':0.2})
    filenames=sorted(parameter_dict.keys())
    for i,filename in enumerate(filenames):
        parameters=parameter_dict[filename]
        img=cv2.imread(f"images/{filename}")
        img=cv2.cvtColor(img, cv2.COLOR_BGR2RGB)        
        y,x=divmod(i,num_columns)
        ax=axes[y][x]
        ax.grid(None)
        ax.axis("off")
        ax.imshow(img)
        ax.set_title(f"light:{int(parameters['z_light']):<4} bunny:{int(parameters['z_bunny']):<4}")
    fig.set_size_inches((scale*num_rows,scale*num_columns))
    plt.show()
    
with open("todo2.json","r") as fp:
    parameter_dict=json.load(fp)
show_images(parameter_dict,scale=3)

In [ ]:
# After verifying that the images above have the correct labels as printed in the grid
# run this cell
torch.manual_seed(148)
rng = np.random.default_rng(148)

#load images in a way our optimization framework can read
def load_single_image(filename):
    image=cv2.imread(f"images/{filename}")
    image=torch.from_numpy(np.stack([image],axis=0).transpose(0,3,1,2)).float()/255.0 -0.5
    return image
def load_images_and_z_angles(filenames:str, param_dict):
    images=[cv2.imread(f"images/{filename}") for filename in filenames]
    z_angles=[[param_dict[f]["z_light"],param_dict[f]["z_bunny"]] for f in filenames]
    images=torch.from_numpy(np.stack(images,axis=0).transpose(0,3,1,2)).float()/255.0 -0.5
    z_angles=torch.Tensor(z_angles)/180.
    return images, z_angles

# define the problem: find G that minimizes ||G(yi)-xi||, 
# where yi is our observed images, and xi is our observed (z_light, z_bunny).
# once we find a G that can predict xi from yi, 
# we can try predicting additional images that weren't previously observed!

#setup a parametrization for G
G = torch.nn.Sequential(
    torch.nn.Conv2d(3, 4, 3, padding=1, stride=2),
    torch.nn.ReLU(),
    torch.nn.Conv2d(4, 4, 3, padding=1, stride=2),
    torch.nn.ReLU(),
    torch.nn.Conv2d(4, 8, 3, padding=1, stride=2),
    torch.nn.ReLU(),
    torch.nn.Conv2d(8, 8, 3, padding=1, stride=2),
    torch.nn.ReLU(),
    torch.nn.Flatten(),
    torch.nn.Linear(512, 8),
    torch.nn.ReLU(),
    torch.nn.Linear(8, 2),
    torch.nn.Sigmoid()
)

distance_fn = torch.nn.MSELoss(reduction='sum')
solver = torch.optim.RMSprop(G.parameters(), lr=10**(-3))

num_images = len(parameter_dict)
filenames=list(parameter_dict.keys())

average_dists=[]
for t in tqdm.notebook.tqdm(range(100)):
    random_order=np.arange(num_images)
    rng.shuffle(random_order)
    random_order=np.array_split(random_order,1)
    dists=[]
    for batch in random_order:
        #load in some xi, yi
        batch_filenames=[filenames[i] for i in batch]
        images, poses=load_images_and_z_angles(batch_filenames, parameter_dict)
        # Compute G(yi)
        pose_pred = G(images)
        # Compute ||G(yi)-xi||
        dist = distance_fn(pose_pred, poses)
        # Minimize ||G(yi)-xi|| using optimization techniques to walk down the hill (gradient descent)
        # Set initial downhill direction to zero
        solver.zero_grad()
        # Find downhill direction: compute gradient of the distance with respect to parameters of G
        dist.backward()
        # Step in some direction that we think takes us downhill
        solver.step()
        dists.append(dist.item())
    average_dists.append(np.mean(dists))

In [ ]:
with torch.no_grad():
    unseen_image=load_single_image("todo1.png")
    unseen_pose=G(unseen_image)[0].numpy()*180
    prediction={"predicted_z_light":unseen_pose[0], "predicted_z_bunny":unseen_pose[1]}
    print(prediction)
    print("load this prediction into Blender's TODO5!")

In [ ]:
# run this cell after doing TODO5 in Blender
show_image("images/todo1.png",label="original image y")
show_image("images/todo5.png",label="render corresponding to pose from G(y)")

In [ ]:
# SAVE THE SCRIPT IN BLENDER (Text -> Save or Alt/Option+S), 
# THEN RUN THIS CELL BEFORE SUBMITTING.
# READ THE LOADED CODE AND MAKE SURE ALL YOUR TODOs HAVE BEEN UPDATED !!
%load hw5_blender.py